# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/monajam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/monajam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '23604d'. Skipping!
Property 'summary' already exists in node '2946eb'. Skipping!
Property 'summary' already exists in node '2f2645'. Skipping!
Property 'summary' already exists in node '337109'. Skipping!
Property 'summary' already exists in node 'a0e13f'. Skipping!
Property 'summary' already exists in node 'b330e1'. Skipping!
Property 'summary' already exists in node '4f0811'. Skipping!
Property 'summary' already exists in node '027e70'. Skipping!
Property 'summary' already exists in node '3c8b9b'. Skipping!
Property 'summary' already exists in node 'e28e1c'. Skipping!
Property 'summary' already exists in node '008454'. Skipping!
Property 'summary' already exists in node '0b3fb6'. Skipping!
Property 'summary' already exists in node '28f623'. Skipping!
Property 'summary' already exists in node '8554ad'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '337109'. Skipping!
Property 'summary_embedding' already exists in node '2946eb'. Skipping!
Property 'summary_embedding' already exists in node '28f623'. Skipping!
Property 'summary_embedding' already exists in node '027e70'. Skipping!
Property 'summary_embedding' already exists in node '2f2645'. Skipping!
Property 'summary_embedding' already exists in node '3c8b9b'. Skipping!
Property 'summary_embedding' already exists in node 'b330e1'. Skipping!
Property 'summary_embedding' already exists in node '4f0811'. Skipping!
Property 'summary_embedding' already exists in node '23604d'. Skipping!
Property 'summary_embedding' already exists in node 'e28e1c'. Skipping!
Property 'summary_embedding' already exists in node '008454'. Skipping!
Property 'summary_embedding' already exists in node 'a0e13f'. Skipping!
Property 'summary_embedding' already exists in node '0b3fb6'. Skipping!
Property 'summary_embedding' already exists in node '8554ad'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
Simple fact-based queries from one chunk

Conceptual/multi-step queries drawing from multiple nodes

Multi-step, detailed queries using overlapping entities or keyphrases

In details:

1-This synthesizer creates specific, direct questions based on a single chunk/node of a knowledge graph.

2-This one builds abstract (high-level, conceptual) queries that link multiple nodes or chunks in the graph.

3-This synthesizer constructs specific multi-hop questions, leveraging overlapping entities or keyphrases across multiple chunks.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How is the academic year defined for different...,"[Chapter 1 Academic Years, Academic Calendars,...",The academic year must be defined for every el...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) specify regarding we...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,"According to Chapter 3, under what conditions ...",[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What is the role of the Federal Work-Study pro...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,What is Volume 7 about in the context of feder...,[both the credit or clock hours and the weeks ...,Volume 7 provides guidance on the credit or cl...,single_hop_specifc_query_synthesizer
5,separate academic years for different program ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that a school can define ...,multi_hop_abstract_query_synthesizer
6,How do the defined payment periods relate to t...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The payment periods are established within the...,multi_hop_abstract_query_synthesizer
7,How does the variation in academic year defini...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The variation in academic year definitions for...,multi_hop_abstract_query_synthesizer
8,Volume 2 and Volume 8 what do they say about a...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 explains that academic years must hav...,multi_hop_specific_query_synthesizer
9,How do Volume 2 and Volume 7 relate to academi...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the academic year requireme...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '4ae042'. Skipping!
Property 'summary' already exists in node '6ddb24'. Skipping!
Property 'summary' already exists in node 'd60b09'. Skipping!
Property 'summary' already exists in node '9cc5c4'. Skipping!
Property 'summary' already exists in node 'd6d439'. Skipping!
Property 'summary' already exists in node '915422'. Skipping!
Property 'summary' already exists in node '50b61f'. Skipping!
Property 'summary' already exists in node 'd0452e'. Skipping!
Property 'summary' already exists in node 'b39fc4'. Skipping!
Property 'summary' already exists in node '04c225'. Skipping!
Property 'summary' already exists in node 'd3c46b'. Skipping!
Property 'summary' already exists in node '771dea'. Skipping!
Property 'summary' already exists in node '600f88'. Skipping!
Property 'summary' already exists in node 'f49792'. Skipping!


...

Property 'summary_embedding' already exists in node '6ddb24'. Skipping!
Property 'summary_embedding' already exists in node 'd60b09'. Skipping!
Property 'summary_embedding' already exists in node '9cc5c4'. Skipping!
Property 'summary_embedding' already exists in node '4ae042'. Skipping!
Property 'summary_embedding' already exists in node 'd6d439'. Skipping!
Property 'summary_embedding' already exists in node '915422'. Skipping!
Property 'summary_embedding' already exists in node 'b39fc4'. Skipping!
Property 'summary_embedding' already exists in node '771dea'. Skipping!
Property 'summary_embedding' already exists in node '50b61f'. Skipping!
Property 'summary_embedding' already exists in node 'd0452e'. Skipping!
Property 'summary_embedding' already exists in node 'f49792'. Skipping!
Property 'summary_embedding' already exists in node '04c225'. Skipping!
Property 'summary_embedding' already exists in node 'd3c46b'. Skipping!
Property 'summary_embedding' already exists in node '600f88'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Chapter 2 about?,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not include specific...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(b) in relation to weeks o...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What information does Volume 8 provide regardi...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 states that clinical work conducted o...,single_hop_specifc_query_synthesizer
3,What is the significance of Title IV in relati...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Inclusion of clinical work in standard term pe...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
5,How guidance and exceptions related to clinica...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in a standard t...,multi_hop_abstract_query_synthesizer
6,disbursement timing in subscription programs a...,[<1-hop>\n\nboth the credit or clock hours and...,"In the first two subscription periods, student...",multi_hop_abstract_query_synthesizer
7,How does the impact of additonal hours or week...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in clock-hour or non...,multi_hop_abstract_query_synthesizer
8,How does Volume 8 relate to the inclusion of c...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 8 provides guidance on including clinic...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B relate to dis...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (student Federal PLUS Loans)  \n- Federal Stafford Loans (Subsidized and Unsubsidized)  \n- Federal SLS Loans  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)\n\nNote: Federal Family Education Loan (FFEL) Program loans (such as Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans) were made before the authority to make new FFEL Program loans ended effective July 1, 2010. New loans are made under the Direct Loan Program.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

###Evaluates: Correctness of the model's answer (Question-Answer quality).

How it works: Compares the model’s generated answer to a reference answer.

###Evaluates: Helpfulness of the answer based on a labeled reference.

How it works: Uses a custom criterion (“helpfulness”) and compares the model’s response to the reference answer, considering whether it helps the user.

###Evaluates: Empathy of the response.

How it works: Judges whether the model’s answer acknowledges the user's feelings or provides a compassionate tone.

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'spotless-parcel-84' at:
https://smith.langchain.com/o/fbdffdce-5f8b-5660-ad4e-686bfc2dced2/datasets/f3da0256-9994-4283-9bd9-f627495bb80f/compare?selectedSessions=f9717afc-dac3-4ac0-a281-cde98da1ed84




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively inform th...,Chapters 2 and 3 collectively provide detailed...,None,Chapter 3 explains that clinical work conducte...,1,1,0,9.422122,c4690d21-a7f5-495b-83c2-010e4f1f183d,b15e3b50-1fd9-4199-9890-842a82237c1b
1,"In Volume 2 and Volume 8, how does the academi...","Based on the provided context, Volume 2 (Acade...",None,"According to Volume 2, the academic year must ...",1,1,0,6.260916,756b727b-3fa4-4cb1-91f9-c51af33d808b,2f2304f7-0d97-42f4-900e-bef57e0cf410
2,How do Appendix A and Appendix B relate to dis...,"Based on the provided context, Appendix B (Vol...",None,Appendix B provides detailed guidance on disbu...,0,0,0,7.808201,176946f7-11eb-4bcd-b66b-4664f035671d,82aefc9f-2291-4efd-acf5-673091b3408f
3,How does Volume 8 relate to the inclusion of c...,"Based on the provided context, Volume 8 (The D...",None,Volume 8 provides guidance on including clinic...,1,0,0,3.729049,9af5d8b1-26b7-49a8-b3c0-20171ae58bec,1ca295e2-1d73-453b-a100-ce2c8a82f253
4,How does the impact of additonal hours or week...,Based on the provided context:\n\nIn clock-hou...,None,The context explains that in clock-hour or non...,1,1,0,8.406973,39a0058f-a3d6-4556-b049-d5ab919828de,10fc8620-dcf2-4941-984f-36ce8bffa855
5,disbursement timing in subscription programs a...,"In subscription-based programs, for the first ...",None,"In the first two subscription periods, student...",1,1,0,5.789038,3d18af06-ddd5-4b5d-9b93-7ffe05d7fdf7,902a6038-3281-4874-9c25-917b27461aa2
6,How guidance and exceptions related to clinica...,"Based on the provided context, clinical work i...",None,The inclusion of clinical work in a standard t...,1,1,0,6.240570,ac0b3645-6365-4044-adac-e4307764dcc5,f3671755-699c-4a81-99b3-8a7673bac81d
7,Inclusion of clinical work in standard term pe...,Based on the context provided:\n\n- Clinical w...,None,The inclusion of clinical work in standard ter...,1,1,0,5.544271,c1498f6c-26f6-4315-b0dc-cb4a3a62aee3,fe5c6f66-5ea2-4e8f-a2e9-ec48275bb0ca
8,What is the significance of Title IV in relati...,The significance of Title IV in relation to pa...,None,The payment period is applicable to all Title ...,1,1,0,2.686657,457a4159-e928-474d-bb3b-af25944a6685,fc975419-9944-48ab-84c7-ce9eea1f650b
9,What information does Volume 8 provide regardi...,"Volume 8, Chapter 3 provides additional guidan...",None,Volume 8 states that clinical work conducted o...,0,0,0,1.173017,eaa8cfb1-2bff-4633-8ad7-684881d90c14,68d5f422-57c6-4024-a79f-1897f472f1c9


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

🚀 Too Small: Chunks might miss important context.

🚀 Too Large: Chunks might contain irrelevant or noisy information, confusing the retriever or exceeding token limits.

✅ With a good chunk size, the retriever can find just enough context to answer the query well

🚀 Smaller chunks → More chunks → Higher embedding & storage cost.
Larger chunks → Fewer chunks, but possibly wasted tokens on unrelated info.

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

Better models (like text-embedding-3-large) understand deeper relationships, synonyms, and abstract ideas

High-quality embeddings = Better similarity matches between query and documents.

Larger models often produce higher-dimensional vectors (e.g., 1536 vs. 3072 dimensions), which:

Improve accuracy,But increase storage, search time, and cost

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question! Based on the information provided in the context, there are several types of federal student loans available:\n\n1. **Direct Subsidized Loans** – These loans are for students with financial need, and the government pays the interest while you are in school.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need, and interest accrues while you are in school.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to help pay for their child’s cost of attendance. Graduate or professional students can also apply for Direct PLUS Loans. There is no fixed limit for PLUS loans, but they cannot exceed the student’s cost of attendance minus other aid.\n\nAdditionally, if a dependent student’s parent cannot get a Direct PLUS Loan, the student may be eligible for additional Direct Unsubsidized Loan funds, sometimes up to the amounts available for independent students.\n\nI hope th

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'whispered-snail-25' at:
https://smith.langchain.com/o/fbdffdce-5f8b-5660-ad4e-686bfc2dced2/datasets/f3da0256-9994-4283-9bd9-f627495bb80f/compare?selectedSessions=48922fdc-2708-4dca-ab5e-1e1a329120d5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively inform th...,Thank you for your thoughtful question. Based ...,None,Chapter 3 explains that clinical work conducte...,1,1,1,4.098360,c4690d21-a7f5-495b-83c2-010e4f1f183d,fb584c59-5716-42d9-b709-6557db5ddf05
1,"In Volume 2 and Volume 8, how does the academi...",Thank you for your thoughtful question. Based ...,None,"According to Volume 2, the academic year must ...",1,1,1,3.732316,756b727b-3fa4-4cb1-91f9-c51af33d808b,e9e449a5-1127-4f96-8b07-b52c49b0cc0d
2,How do Appendix A and Appendix B relate to dis...,Thank you for your thoughtful question. It’s c...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.683485,176946f7-11eb-4bcd-b66b-4664f035671d,baea701b-6c53-4e67-98b4-6dd29f745483
3,How does Volume 8 relate to the inclusion of c...,Thank you for your thoughtful question. Based ...,None,Volume 8 provides guidance on including clinic...,1,0,1,2.962966,9af5d8b1-26b7-49a8-b3c0-20171ae58bec,c6891e5a-889b-4be6-8e2e-d344e92d0ef1
4,How does the impact of additonal hours or week...,Thank you for your thoughtful question. I unde...,None,The context explains that in clock-hour or non...,1,1,1,6.350770,39a0058f-a3d6-4556-b049-d5ab919828de,7248a51d-e2c2-44ff-ac7a-c04dcabc6b19
5,disbursement timing in subscription programs a...,Thank you for your thoughtful question. It sou...,None,"In the first two subscription periods, student...",1,1,1,4.114755,3d18af06-ddd5-4b5d-9b93-7ffe05d7fdf7,0e9854a5-f592-4b89-bdae-ddc7f2236ff3
6,How guidance and exceptions related to clinica...,Thank you for your thoughtful question. Based ...,None,The inclusion of clinical work in a standard t...,1,0,1,4.805241,ac0b3645-6365-4044-adac-e4307764dcc5,910ad2b6-3650-4e0b-8a57-4d063177bee8
7,Inclusion of clinical work in standard term pe...,Thank you for your thoughtful question about h...,None,The inclusion of clinical work in standard ter...,1,1,1,5.219192,c1498f6c-26f6-4315-b0dc-cb4a3a62aee3,be993d3e-1c7f-492b-89a1-67d4f6a9b24f
8,What is the significance of Title IV in relati...,Thank you for your thoughtful question. Based ...,None,The payment period is applicable to all Title ...,1,1,1,2.555724,457a4159-e928-474d-bb3b-af25944a6685,3d0871f2-7c35-40c4-927b-fefb02a2934c
9,What information does Volume 8 provide regardi...,Thank you for your thoughtful question. Based ...,None,Volume 8 states that clinical work conducted o...,1,1,1,2.644281,eaa8cfb1-2bff-4633-8ad7-684881d90c14,9b836416-97a9-4868-9f26-f986e2834b12


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![My Research Graph](pics/pic1.png)
![My Research Graph](pics/pic2.png)
![My Research Graph](pics/pic3.png)
![My Research Graph](pics/pic4.png)
![My Research Graph](pics/pic5.png)